# Import Required Libraries
Import all necessary Python libraries, including requests, BeautifulSoup, sqlite3, datetime, os, and logging.

In [2]:
!pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import requests
from bs4 import BeautifulSoup, Tag
from datetime import datetime
import sqlite3
import os
import logging

# Define Constants and Configuration
Set up constants such as the target URL, database path, table name, and category names. Configure logging and ensure the data directory exists.

In [4]:
# Constants and configuration
URL = "https://www.riigiteataja.ee/akt/128122024049"
DB_PATH = "data/narcotics.db"
CATEGORIES = {
    "I NIMEKIRI", "II NIMEKIRI", "III NIMEKIRI",
    "IV NIMEKIRI", "V NIMEKIRI", "VI NIMEKIRI"
}
TABLE_NAME = "narcotics"

os.makedirs("data", exist_ok=True)
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Fetch HTML Content from URL
Write a function to fetch HTML content from the specified URL using requests, with error handling and logging.

In [5]:
def fetch_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        logging.error(f"Failed to fetch URL: {e}")
        raise

# Set Up SQLite Database
Create a function to set up the SQLite database, create the required table if it doesn't exist, and clear any existing data.

In [6]:
def setup_db(cur):
    cur.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            category TEXT,
            drug_name TEXT,
            english_name TEXT,
            collected_on TEXT
        )
    """)
    # Clear the table before inserting new data
    cur.execute(f"DELETE FROM {TABLE_NAME}")
    logging.info(f"Table '{TABLE_NAME}' cleared before new insertions.")

# Parse HTML and Insert Data into Database
Write a function to parse the HTML using BeautifulSoup, extract the relevant data from tables, and insert it into the database.

In [7]:
def parse_and_insert(soup, cur, today):
    insert_count = 0
    # Normalize categories for matching
    normalized_categories = {c.strip().upper().replace("\xa0", " ").replace("&nbsp;", " "): c for c in CATEGORIES}
    current_category = None
    # Flatten all elements in body in order
    body_elements = list(soup.body.descendants)
    for elem in body_elements:
        # Look for <b> tags inside <p> for category headers
        if isinstance(elem, Tag) and elem.name == "p":
            b_tags = elem.find_all("b")
            for b_tag in b_tags:
                found = b_tag.get_text(separator=" ", strip=True).upper().replace("\xa0", " ").replace("&nbsp;", " ")
                found = " ".join(found.split())
                if found in normalized_categories:
                    current_category = normalized_categories[found]
        # If a table is found and we have a current category, process it
        if isinstance(elem, Tag) and elem.name == "table" and current_category:
            rows = elem.find_all("tr")[1:]  # Skip header
            for row in rows:
                cols = row.find_all("td")
                if len(cols) >= 2:
                    estonian = cols[0].get_text(separator=" ", strip=True)
                    english = cols[1].get_text(separator=" ", strip=True)
                    if estonian:
                        try:
                            cur.execute(f"""
                                INSERT INTO {TABLE_NAME} (category, drug_name, english_name, collected_on)
                                VALUES (?, ?, ?, ?)
                            """, (current_category, estonian, english, today))
                            insert_count += 1
                        except sqlite3.DatabaseError as db_err:
                            logging.error(f"DB insert error: {db_err}")
            # Do NOT reset current_category here; next table may belong to same category unless a new header is found
    logging.info(f"Inserted {insert_count} rows into '{TABLE_NAME}'.")
    return insert_count

# Run the Scraping Workflow
Combine the previous steps to fetch the HTML, parse it, and insert the data into the database. Commit the transaction and log the results.

In [8]:
# Run the scraping workflow
html = fetch_html(URL)
soup = BeautifulSoup(html, "html.parser")
today = datetime.now().strftime("%Y-%m-%d")
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    setup_db(cur)  # Ensures table exists and clears it
    inserted = parse_and_insert(soup, cur, today)
    conn.commit()
print(f"Inserted {inserted} rows into '{TABLE_NAME}'.")

INFO: Table 'narcotics' cleared before new insertions.
INFO: Inserted 425 rows into 'narcotics'.
INFO: Inserted 425 rows into 'narcotics'.


Inserted 425 rows into 'narcotics'.


# Verify Inserted Data
Query the database to display a sample of the inserted data and verify that the scraping and insertion were successful.

In [9]:
# Preview a few rows from the database to verify
import pandas as pd
with sqlite3.connect(DB_PATH) as conn:
    df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME} LIMIT 10", conn)
display(df)

,id,category,drug_name,english_name,collected_on
0,1854,I NIMEKIRI,1-(1-bensofuraan-5-üül)-N-etüülpropaan-2-amiin...,1-(1-benzofuran-5-yl)-N-ethylpropan-2-amine (5...,2025-07-10
1,1855,I NIMEKIRI,"1-(4-bromo-2,5-dimetoksüfenüül)-2-aminoetaan (...","1-(4-bromo-2,5-dimethoxyphenethylamine; (2C-B;...",2025-07-10
2,1856,I NIMEKIRI,1-(bensofuraan-5-üül)-N-metüülpropaan-2-amiin ...,1-(benzofuran-5-yl)-N-methylpropan-2-amine (5-...,2025-07-10
3,1857,I NIMEKIRI,"1B-LSD\n (4-butürüül-N,N-dietüül-7-metüül 4,6...","1B-LSD\n (4-butyryl-N,N-diethyl-7-methyl- 4,6...",2025-07-10
4,1858,I NIMEKIRI,1cP-LSD\n (Curie Acid; Curie); 1- tsüklopropi...,1cP-LSD (Curie Acid; Curie); 1- cyclopropionyl...,2025-07-10
5,1859,I NIMEKIRI,1-fenüületüülamiin(1-PEA);1- fenüületaanamiin;...,1-phenylethylamine(1-PEA);1- phenylethanamine;...,2025-07-10
6,1860,I NIMEKIRI,1-naftalenüül(1-pentüül-1H-indasool-3-üül)- me...,1-naphthalenyl(1-pentyl-1H-indazol-3-yl)- meth...,2025-07-10
7,1861,I NIMEKIRI,"1-propionüül-lüsergiinhappedietüülamiid;N,N- d...","1-propionyl-lysergicaciddiethylamide;N,N- diet...",2025-07-10
8,1862,I NIMEKIRI,"1V-LSD(N,N-dietüül-7-metüül-4-pentanoüül- 4,6,...","1V-LSD(N,N-diethyl-7-methyl-4-pentanoyl- 4,6,6...",2025-07-10
9,1863,I NIMEKIRI,"2-(4-bromo-2,5-dimetoksüfenüül)-N-[(2- metoksü...","2-(4-bromo-2,5-dimethoxyphenyl)-N-[(2- methoxy...",2025-07-10


# Preview All Unique Categories in the Database
Check that all categories (I–VI) are present after scraping.

In [10]:
with sqlite3.connect(DB_PATH) as conn:
    df_cat = pd.read_sql_query(f"SELECT DISTINCT category FROM {TABLE_NAME}", conn)
display(df_cat)

,category
0,I NIMEKIRI
1,II NIMEKIRI
2,III NIMEKIRI
3,IV NIMEKIRI
4,V NIMEKIRI
5,VI NIMEKIRI
